# Lab 11- Extended Exercises on Time Series Clustering

You are the Senior Data Scientist in a learning platform called LernTime. Your data science team built a data frame in which each row contains the aggregated features per student (calculated over the first 5 weeks of interactions) and the feature `dropout` indicates whether the student stopped using the platform (1) or not (0) before week 10.

The dataframe is in the file `lerntime.csv` and contains the following features:
- `video_time`: total video time (in minutes) 
- `num_sessions` total number of sessions
- `num_quizzes`: total number of quizzes attempts
- `reading_time`: total theory reading time
- `previous_knowledge`: standardized previous knowledge
- `browser_speed`: standardized browser speed
- `device`:  whether the student logged in using a smartphone (1) or a computer (-1)
- `topics`: the topics covered by the user
- `education`: current level of education (0: middle school, 1: high school, 2: bachelor, 3: master, 4: Ph.D.).
- `dropout`: whether the student stopped using the platform (1) or not (0) before week 5.

In [1]:
import pandas as pd

# Data directory
DATA_DIR = "./../../data/"

In [2]:
df = pd.read_csv(f'{DATA_DIR}/lerntime_dropout.csv')

In [3]:
df.head()

,video_time,num_sessions,num_quizzes,reading_time,previous_knowledge,browser_speed,device,topics,education,dropout
0,45.793303,99.0,36.0,48.186562,1.675972,-0.294704,1.0,"['Locke', 'Descartes', 'Socrates', 'Kant', 'Ni...",2.0,0
1,51.331242,57.0,12.0,49.945810,0.700522,1.253694,1.0,"['Nietzche', 'Locke', 'Confucius', 'Aristotle'...",3.0,0
2,87.414834,52.0,7.0,20.611978,1.836716,-1.171352,1.0,"['Plato', 'Locke', 'Nietzche', 'Socrates', 'De...",4.0,0
3,58.556388,47.0,31.0,33.785805,0.209577,-2.043047,1.0,"['Aristotle', 'Socrates', 'Plato', 'Confucius'...",3.0,0
4,74.822362,58.0,37.0,38.907983,0.265678,-0.754559,1.0,"['Kant', 'Aristotle', 'Confucius', 'Locke', 'P...",4.0,0


In [4]:
# df row count
df.shape[0]

300

You decide to explore the different type of users. You want to use your knowledge from your ML4BD course and decide to cluster using Spectral Clustering. 
In the course, you learnt different ways of constructing the similarity graph, yielding the adjacency matrix serving as an input to the Spectral Clustering. 
Based on your in-depth exploration of the data, you decide to construct the similarity graph as a  *k-nearest neighbor graph*.

Your tasks are to:

a) Write a function to compute the k-nearest neighbor graph.

b) Cluster the users using Spectral Clustering and your k-nearest neighbor graph function (use 4 neighbors). Use only the features *reading_time* and *topics*. You can assume that optimal number of clusters is 2.


## a) Computation of the k-nearest neighbor graph 
Unfortunately, there is no k-nearest neighbor graph implementation available in scikit-learn and you therefore have to implement the function yourself.

The function `'k_nearest_neighbor_graph'` takes a similarity matrix `S` as well as the number of neighbors `k` as an input an returns the adjacency matrix `W`.

Note that we will not evaluate the coding efficiency of your function. 

In [5]:
import numpy as np


def k_nearest_neighbor_graph(S, k):
    # S: similarity matrix
    # k: number of neighbors
    np_S = np.array(S)
    # For each entry in S, keep the k+1 largest values and set the rest to 0
    # we do k+1 because the largest value will be the entry itself and we don't want to consider it as a neighbor
    indexes = np.argsort(np_S, axis=1)[:, -(k+1):]
    print("Indexes shape: ", indexes.shape) # For each row we get the indexes of the k+1 largest values
    # To access a specific row,col of a numpy array we can use the following syntax:
    # np_S[row, col]
    # To access multiple elements per row we can use fancy indexing:
    # np_S[row, [col1, col2, col3]]
    # What we want to achieve is to access indexes[0] for row 0, indexes[1] for row 1, etc.
    # To do this we can use:
    # np.arange(np_S.shape[0])[:, None]
    # This will create a 2D array with the shape (n, 1), and indxes has shape (n, k+1)
    print("arange shape: ", np.arange(np_S.shape[0])[:, None].shape)
    W = np.zeros_like(S)
    W[np.arange(np_S.shape[0])[:, None], indexes] = np_S[np.arange(np_S.shape[0])[:, None], indexes]
    # Make it symmetric
    W = np.maximum(W, W.T)
    return W

In [6]:
from sklearn.neighbors import kneighbors_graph
def k_nearest_neighbor_graph_from_lecture(S, k):
    # S: similarity matrix
    # k: number of neighbors
   
    S = np.array(S)
    # k+1 because include_self. -S to pass from similarity to distance, +translation to avoid negative values
    G = kneighbors_graph(-S + S.max(), k+1, metric='precomputed', mode='connectivity', include_self=True).toarray()
    W = (G + G.T).astype(bool) * S
    
    return W

In [7]:
# What np.argsort does is that it returns the indexes of the sorted array
# i.e if we have [1, 3, 2] it will return [0, 2, 1]. Which means that the smallest element is at index 0, the second smallest at index 2 and the largest at index 1
# If we get the last k elements of this array, we get THE INDEXES of k largest elements of the original array

k = 2
# Please run this cell for evaluation purposes
S = [
    [1, 0.2, 0.7, 0.1],
    [0.2, 1, 0.8, 0.4],
    [0.7, 0.8, 1, 0.6],
    [0.1, 0.4, 0.6, 1]
]

a = k_nearest_neighbor_graph(S, k)
print(a)
b = k_nearest_neighbor_graph_from_lecture(S, k)
print(b)
#send(a, 1)

Indexes shape:  (4, 3)
arange shape:  (4, 1)
[[1.  0.2 0.7 0. ]
 [0.2 1.  0.8 0.4]
 [0.7 0.8 1.  0.6]
 [0.  0.4 0.6 1. ]]
[[1.  0.2 0.7 0. ]
 [0.2 1.  0.8 0.4]
 [0.7 0.8 1.  0.6]
 [0.  0.4 0.6 1. ]]


In [8]:
# Please run this cell for evaluation purposes
k = 2
S = [[1, 0.3, 0.01, 0.1],
     [0.3, 1, 0.8, 0.9],
     [0.01, 0.8, 1, 0.6],
     [0.1, 0.9, 0.6, 1]]
a = k_nearest_neighbor_graph(S, k)
print(a)

Indexes shape:  (4, 3)
arange shape:  (4, 1)
[[1.  0.3 0.  0.1]
 [0.3 1.  0.8 0.9]
 [0.  0.8 1.  0.6]
 [0.1 0.9 0.6 1. ]]


## b) Spectral Clustering 
Perform a spectral clustering using a k-nearest neighbor graph (with 4 neighbors). 

Use the two features `reading_time` and `topics` only. 

If you did not manage to solve task a), use a *fully connected graph* as similarity graph to obtain the adjacency matrix `W`. 

You can assume that the optimal number of clusters is 2. 

Print the obtained cluster labels. 

In [9]:
from sklearn.manifold import spectral_embedding
from sklearn.cluster import KMeans
from scipy import linalg
from scipy.sparse.csgraph import laplacian

def spectral_clustering(W, n_clusters, random_state=111):
    """
    Spectral clustering
    :param W: np array of adjacency matrix
    :param n_clusters: number of clusters
    :param normed: normalized or unnormalized Laplacian
    :return: tuple (kmeans, proj_X, eigenvals_sorted)
        WHERE
        kmeans scikit learn clustering object
        proj_X is np array of transformed data points
        eigenvals_sorted is np array with ordered eigenvalues 
        
    """
    # Compute eigengap heuristic
    L = laplacian(W, normed=True)
    eigenvals, _ = linalg.eig(L)
    eigenvals = np.real(eigenvals)
    eigenvals_sorted = eigenvals[np.argsort(eigenvals)]

    # Create embedding
    random_state = np.random.RandomState(random_state)
    proj_X = spectral_embedding(W, n_components=n_clusters,
                              random_state=random_state,
                              drop_first=False)

    # Cluster the points using k-means clustering
    kmeans = KMeans(n_clusters=n_clusters, random_state = random_state)
    kmeans.fit(proj_X)

    return kmeans, proj_X, eigenvals_sorted

In [10]:
reading_time = df['reading_time'].values.reshape(-1,1)
topics = df['topics'].values
topics_np = np.array([(eval(t)) for t in topics])
# unique topics 
topics_unique = set([item for sublist in topics_np for item in sublist])
print(len(topics_unique))
print(topics_np.shape)
print(reading_time.shape)

8
(300,)
(300, 1)


/var/folders/75/94jyvgc92_gcwmzgrg36xfd00000gn/T/ipykernel_6177/1798036342.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  topics_np = np.array([(eval(t)) for t in topics])


In [11]:
# do one hot encoding for topics
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
topics_one_hot = mlb.fit_transform(topics_np)
print(topics_one_hot.shape)

(300, 8)


In [12]:
# Reading time similarity matrix we can use Gaussian kernel use pairwise_kernels from sklearn.metrics.pairwise
from sklearn.metrics.pairwise import pairwise_kernels
reading_similarity = pairwise_kernels(reading_time, metric='rbf')
print(reading_similarity.shape)

(300, 300)


In [13]:
reading_similarity

array([[1.00000000e+000, 4.52770721e-002, 0.00000000e+000, ...,
        3.15125400e-034, 5.24800016e-162, 0.00000000e+000],
       [4.52770721e-002, 1.00000000e+000, 0.00000000e+000, ...,
        3.76060768e-022, 6.64921189e-134, 0.00000000e+000],
       [0.00000000e+000, 0.00000000e+000, 1.00000000e+000, ...,
        0.00000000e+000, 0.00000000e+000, 7.88239994e-001],
       ...,
       [3.15125400e-034, 3.76060768e-022, 0.00000000e+000, ...,
        1.00000000e+000, 1.69972568e-048, 0.00000000e+000],
       [5.24800016e-162, 6.64921189e-134, 0.00000000e+000, ...,
        1.69972568e-048, 1.00000000e+000, 0.00000000e+000],
       [0.00000000e+000, 0.00000000e+000, 7.88239994e-001, ...,
        0.00000000e+000, 0.00000000e+000, 1.00000000e+000]])

In [14]:
topics_from_class = df[['topics']].apply(lambda x: set(eval(x.topics)), axis=1).to_numpy().reshape(-1, 1)

In [15]:
from scipy.spatial.distance import pdist, cdist, squareform

test1 = cdist(topics_from_class,topics_from_class, metric=lambda x, y: float(len(x[0].intersection(y[0])) / len(x[0].union(y[0]))))
test2 = squareform(pdist(topics_from_class, metric=lambda x, y: 1 - (float(len(x[0].intersection(y[0])) / len(x[0].union(y[0]))))))

print(test1.shape)
print(test2.shape)
print(np.allclose(test1, 1-test2))

(300, 300)
(300, 300)
True


In [20]:
test3 = squareform(pdist(topics_one_hot, metric='jaccard'))
print(test3)
print(test3.shape)
print(np.allclose(test2, test3))

[[0.         0.125      0.125      ... 0.25       0.375      0.125     ]
 [0.125      0.         0.         ... 0.14285714 0.5        0.25      ]
 [0.125      0.         0.         ... 0.14285714 0.5        0.25      ]
 ...
 [0.25       0.14285714 0.14285714 ... 0.         0.625      0.375     ]
 [0.375      0.5        0.5        ... 0.625      0.         0.28571429]
 [0.125      0.25       0.25       ... 0.375      0.28571429 0.        ]]
(300, 300)
True


In [26]:
# Topics are sets of strings, we can use Jaccard similarity to compute similarity between topics
from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import pdist
topics_similarity = 1 - pairwise_distances(topics_one_hot, metric='jaccard')
topics_similarity.shape

/opt/homebrew/anaconda3/lib/python3.10/site-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


(300, 300)

In [188]:
topics_similarity

array([[1.        , 0.875     , 0.875     , ..., 0.75      , 0.625     ,
        0.875     ],
       [0.875     , 1.        , 1.        , ..., 0.85714286, 0.5       ,
        0.75      ],
       [0.875     , 1.        , 1.        , ..., 0.85714286, 0.5       ,
        0.75      ],
       ...,
       [0.75      , 0.85714286, 0.85714286, ..., 1.        , 0.375     ,
        0.625     ],
       [0.625     , 0.5       , 0.5       , ..., 0.375     , 1.        ,
        0.71428571],
       [0.875     , 0.75      , 0.75      , ..., 0.625     , 0.71428571,
        1.        ]])

In [189]:
S = (reading_similarity + topics_similarity)/2
S

array([[1.        , 0.46013854, 0.4375    , ..., 0.375     , 0.3125    ,
        0.4375    ],
       [0.46013854, 1.        , 0.5       , ..., 0.42857143, 0.25      ,
        0.375     ],
       [0.4375    , 0.5       , 1.        , ..., 0.42857143, 0.25      ,
        0.76912   ],
       ...,
       [0.375     , 0.42857143, 0.42857143, ..., 1.        , 0.1875    ,
        0.3125    ],
       [0.3125    , 0.25      , 0.25      , ..., 0.1875    , 1.        ,
        0.35714286],
       [0.4375    , 0.375     , 0.76912   , ..., 0.3125    , 0.35714286,
        1.        ]])

In [194]:
k = 4 # why 4 ??
W = k_nearest_neighbor_graph(S, k)

# Class spectral_clustering
clusters = 2
kmeans, proj_X, eigenvals_sorted = spectral_clustering(W, clusters)
y_pred_k_4 = kmeans.labels_
print(y_pred_k_4.shape)
y_pred_k_4

Indexes shape:  (300, 5)
arange shape:  (300, 1)
(300,)


/opt/homebrew/anaconda3/lib/python3.10/site-packages/sklearn/manifold/_spectral_embedding.py:248: UserWarning: Array is not symmetric, and will be converted to symmetric by average with its transpose.
  adjacency = check_symmetric(adjacency)
/opt/homebrew/anaconda3/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


array([0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1,
       1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0,
       0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0], d

In [195]:
k = 2
W = k_nearest_neighbor_graph(S, k)

# Class spectral_clustering
clusters = 2
kmeans, proj_X, eigenvals_sorted = spectral_clustering(W, clusters)
y_pred_k_2 = kmeans.labels_
print(y_pred_k_2.shape)
y_pred_k_2

Indexes shape:  (300, 3)
arange shape:  (300, 1)
(300,)


/opt/homebrew/anaconda3/lib/python3.10/site-packages/sklearn/manifold/_spectral_embedding.py:248: UserWarning: Array is not symmetric, and will be converted to symmetric by average with its transpose.
  adjacency = check_symmetric(adjacency)
/opt/homebrew/anaconda3/lib/python3.10/site-packages/sklearn/manifold/_spectral_embedding.py:274: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(
/opt/homebrew/anaconda3/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], d

In [191]:
k = 4 
W = k_nearest_neighbor_graph_from_lecture(S, k)

# Class spectral_clustering
clusters = 2
kmeans, proj_X, eigenvals_sorted = spectral_clustering(W, clusters)
y_pred = kmeans.labels_
print(y_pred.shape)
y_pred

(300,)


/opt/homebrew/anaconda3/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


array([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], d

In [192]:
W1 = k_nearest_neighbor_graph(S, 4)
W2 = k_nearest_neighbor_graph_from_lecture(S, 4)
print(np.allclose(W1, W2))

Indexes shape:  (300, 5)
arange shape:  (300, 1)
False
